In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/pope/popular/multi/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,label,question_id,category,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a couch in the image?\nAnswer the qu...,"[tensor(1), tensor(0)]","[tensor(1201), tensor(1202)]","[popular, popular]",{'embedding': [[[ 0.00389099 0.00149536 0.00...,"[Yes, Yes, Yes, Yes, Yes, Yes, No, Yes, Yes, N..."
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a truck in the image?\nAnswer the qu...,"[tensor(1), tensor(0)]","[tensor(2517), tensor(2518)]","[popular, popular]",{'embedding': [[[ 0.00389099 0.00149536 0.00...,"[No, Yes, Yes, No, Yes, Yes, No, No, No, No, N..."
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a person in the image?\nAnswer the q...,"[tensor(1), tensor(0)]","[tensor(2391), tensor(2392)]","[popular, popular]",{'embedding': [[[ 0.00389099 0.00149536 0.00...,"[Yes, Yes, Yes, Yes, Yes, Yes, No, No, No, No,..."
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a person in the image?\nAnswer the q...,"[tensor(1), tensor(0)]","[tensor(4441), tensor(4442)]","[popular, popular]",{'embedding': [[[ 0.00389099 0.00149536 0.00...,"[Yes, Yes, Yes, Yes, Yes, Yes, No, No, No, No,..."
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is there a baseball glove in the image?\nAnsw...,"[tensor(1), tensor(0)]","[tensor(585), tensor(586)]","[popular, popular]",{'embedding': [[[ 0.00389099 0.00149536 0.00...,"[Yes, Yes, Yes, Yes, Yes, Yes, No, No, No, No,..."


In [18]:
logits=df['most_likely'].apply(lambda x: x['logits'].cpu()).to_list()

In [19]:
logits=np.concatenate(logits,axis=0)

In [20]:
logits.shape

(5000, 32064)

In [21]:
gt_label=df['label'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [22]:
gt_label.shape

(5000,)

In [23]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [24]:
is_correct=[]
for i,x in enumerate(resps):
    if x.lower()=='no' and gt_label[i]==0:
        is_correct.append(1)
    elif x.lower()=='yes' and gt_label[i]==1:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [25]:
is_correct.shape

(5000,)

In [26]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 5000 responses. 4325 answer correctly and 675 answer wrongly


In [27]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index


'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
y_train = is_correct[train_idxs]
y_test = is_correct[test_idxs]


print(f'trainset length:{len(y_train)}')
print(f'testset length:{len(y_test)}')
print(f'trainset postive: {sum(y_train==1)} negative: {sum(y_train==0)}')
print(
    f'testset postive: {sum(y_test==1)} negative: {sum(y_test==0)}')

trainset length:4000
testset length:1000
trainset postive: 3453 negative: 547
testset postive: 872 negative: 128


In [28]:
logits_train = logits[train_idxs]
logits_test = logits[test_idxs]

Train Linear Prob to Get Better Results

In [30]:
X_train=logits_train
y_train=y_train

X_test=logits_test
y_test=y_test

In [31]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [ ]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.872 AUROC:0.5


: 